Module imports

In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils

from activations import Sigmoid
from losses import binary_cross_entropy, binary_cross_entropy_prime
from dense import Dense
from reshape import Reshape
from convolutional import Convolutional

Helper function

In [2]:
def get_relevant_digits(data: np.ndarray, labels: np.ndarray, digit_1: int, digit_2: int, limit: int) -> np.ndarray:
    # get indexes of digits which correspond to our chosen digits
    digit_1_index = np.where(labels == digit_1)[0][:limit]
    digit_2_index = np.where(labels == digit_2)[0][:limit]

    # combine these together
    all_indices = np.hstack((digit_1_index, digit_2_index))

    # extract the data using the indices
    x, y = data[all_indices], labels[all_indices]

    # convert the digit labels to ones and zeros so binary classification works
    for i, val in enumerate([digit_1, digit_2]):
        y[y == val] = i

    return x, y


def preprocess(x, y):
    # get indexes of digits which correspond to our chosen digits
    zero_index = np.where(y == 0)[0]
    one_index = np.where(y == 1)[0]

    # combine these together
    all_indices = np.hstack((zero_index, one_index))

    # shuffle the array of indices
    all_indices = np.random.permutation(all_indices)

    # extract the data using the indices
    x, y = x[all_indices], y[all_indices]

    # reshape image so it can be inputted into the convolutional layer
    x = x.reshape(len(x), 1, 28, 28)

    # normalise the brightness values
    x = x.astype("float32") / 255

    # use keras to create a one-hot encoding
    y = np_utils.to_categorical(y)

    # rshape to columns so it can be inputtef to the dense layer
    y = y.reshape(len(y), 2, 1)

    return x, y

### Loading the Digit Data

In [3]:
# load the data from keras
(x_train, y_train), (x_test, y_test) = mnist.load_data()

digit_1 = 0
digit_2 = 6

# convert the chosen digits to 1's and 0's
x_train, y_train = get_relevant_digits(x_train, y_train, digit_1, digit_2, 100)
x_train, y_train = get_relevant_digits(x_test, y_test, digit_1, digit_2, 100)

# split sets into data and labels and preprocess them
x_train, y_train = preprocess(x_train, y_train)
x_test, y_test = preprocess(x_test, y_test)

print(f"There are {len(x_train)} digits in the training set, and {len(x_test)} digits in the test set - each with size (1, 28, 28)")

There are 200 digits in the training set, and 2115 digits in the test set - each with size (1, 28, 28)


### Form the Network

In [4]:
network = [
    Convolutional((1, 28, 28), 3, 5),       # here we are using 3 kernels (each are 3x3) and a depth of 5
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)), # reshape into a column
    Dense(5 * 26 * 26, 100),                # dense layer maps it to 100 untis
    Sigmoid(),
    Dense(100, 2),                          # map this to two outputs
    Sigmoid(),
]

### The Training Loop

In [5]:
# some constants
learning_rate = 0.1
epochs = 20

for epoch in range(epochs):

    # reset error to zero for each epoch
    error = 0

    for x, y in zip(x_train, y_train):

        ### ------------------- forward pass
        # set the initial output
        output = x

        # loop through each of the layer in the network (defined above)
        for layer in network:
            output = layer.forward(output)

        ###

        # update the error
        error += binary_cross_entropy(y, output)

        ### ------------------- backward pass
        # set initial gradient
        gradient = binary_cross_entropy_prime(y, output)

        # loop through the network in reverse order
        for layer in reversed(network):
            gradient = layer.backward(gradient, learning_rate)

        ###

    
    error /= len(x_train)
    print(f"{epoch + 1}/{epochs}, error = {error}")

1/20, error = 0.7850467085596696
2/20, error = 0.5295084172801816
3/20, error = 0.4090846741884484
4/20, error = 0.2894849544439524
5/20, error = 0.4426437179138722
6/20, error = 0.3512815958916294
7/20, error = 0.30506530894598527
8/20, error = 0.3926513914329374
9/20, error = 0.2946605111473579
10/20, error = 0.22255364212903228
11/20, error = 0.6830289067590223
12/20, error = 0.6789541393064424
13/20, error = 0.5075549397295377
14/20, error = 0.7390511960131941
15/20, error = 0.5650611047823503
16/20, error = 0.656488416030919
17/20, error = 0.7991237263745739
18/20, error = 0.6417203789802017
19/20, error = 0.5552825511201307
20/20, error = 0.43360834292912825


## Testing

In [6]:
num_correct = 0
for x, y in zip(x_test, y_test):
    output = x
    for layer in network:
        output = layer.forward(output)
    print(f"prediction: {np.argmax(output)}, actual: {np.argmax(y)}")
    if np.argmax(output) == np.argmax(y):
        num_correct += 1

print(f"Percentage of correct: {(num_correct / len(x_test)) * 100}%")

prediction: 0, actual: 1
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 1, actual: 1
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 1, actual: 1
prediction: 1, actual: 1
prediction: 0, actual: 0
prediction: 1, actual: 1
prediction: 1, actual: 1
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 1
prediction: 1, actual: 1
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 1, actual: 1
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 1, actual: 1
prediction: 1, actual: 1
prediction: 0, actual: 1
prediction: 1, actual: 1
prediction: 1, actual: 1
prediction: 0, actual: 1
prediction: 1, actual: 1
prediction: 0, actual: 1
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 0
prediction: 0, actual: 1
prediction: 1, actual: 1
prediction: 0, actual: 0
